# Variational Autoencoder

<div >
    <img src="./data/model_architecturev2.png" alt="Image" width="1000">
</div>

This project explores and implements the ideas of the seminal deep learning paper "Auto-Encoding Variational Bayes". Link to the paper: [arxiv](https://arxiv.org/abs/1312.6114).

### Significance

Why is this paper important?

The paper introduces the Variational Autoencoder (VAE), a powerful model for generative tasks, which involves generating new data samples similar to a given dataset. The paper develops mathematical solutions to the problem of using an autoencoder to generate new samples. Since autoencoders are generally trained in an unsupervised fashion—without relying on explicit data labels—they offer a versatile technique. This is why the can be adapted to a wide range of problems in deep learning. 
VAEs, on their own, are usually not state-of-the-art for specific tasks. However, their adaptability makes them a key building block in larger model architectures. For example, the current state of the art architecture for image generation - Stable Diffusion, relies on VAE as one of its core building blocks.

### Problem Statement

The goal of a VAE model is to learn a distribution of a latent hidden variable $z$ that explains the observed data $x$. There are two key challenges in this:
1. Computing the posterior distribution $p(z|x)$, which is often intractable due to the complexity of the marginal likelihood of the data $p(x)$
2. Efficiently learning model parameters through optimization.

### Proposed Solutions

The authors propose using **variational inference**, a framework for approximating intractable posterior distributions, combined with an efficient reparameterization trick to enable gradient-based optimization. This approach allows for training both the generative model and the approximate posterior simultaneously. The resulting model, the VAE, combines probabilistic modeling with neural networks, making it scalable and efficient.

#### Background

Data that we observe can be thought of as being represented or generated by a hidden latent variable, which cannot be observed directly. An example of this concept can be seen in Plato's "Allegory of the Cave." In Plato's work, a group of people are chained inside a cave and can only see shadows on the wall created by unseen objects passing before a fire. The shadows—a two-dimensional projection—are the data that can be observed, while the hidden three-dimensional objects are the real driving force behind the observations; they are the latent variable.

This latent variable can be either of higher dimensionality, as in Plato's Cave, or of lower dimensionality than the observed data. However, in the field of generative modeling, we generally seek to learn lower-dimensional latent representations. Reducing the dimensionality of the data can be seen as a form of compression and also has the potential to lead to semantically meaningful structures in the latent space in relation to the observed data.

### Key Ideas

#### Evidence Lower Bound - ELBO

Mathematically, the observed data $x$ and the latent variable $z$ can be modeled by a joint distribution:
$$p(x|z)$$
In the case of generative modeling one approach is to learn a model to maximize the likelihood $P(x)$ for all observed data points $x$. This simply means that a trained model should assign high likelihood to real observed data, and low likelihood to everything else. To access the likelihood of the observed data $P(x)$ we could manipulate the joint distribution of observed data and latent variable $P(x|z)$ in two different ways:
*  The first is to marginalize out the latent variable $z$, by integrating the      probability of the observed data across all possible values of $z$:
    $$p(x) = \int p(x, z)dz$$
    The problem is that computing this integral is not possible for more complex models. Some of the reasons for this are potential high dimensionality of the latent space and the lack of closed form solutions of complex functions involving neural networks.

* The second way to get $p(x)$ on its own is to use the chain rule of probability:
  $$P(X_1, X_2, \dots, X_n) = \prod_{i=1}^n P(X_i \mid X_1, X_2, \dots, X_{i-1})$$

    $$p(x,z) = p(x)p(z|x)$$ 
    dividing by $p(z|x)$ we get
    $$p(x) = \frac{p(x,z)}{p(z|x)}$$ 

    This has the problem that it involves having access to ground truth latent encoder $p(z|x)$, which is not available since the latent variable is by definition not directly observable and there is no ground truth mapping from $x$ to $z$. 


The key idea to solve this problems is to use the two equations for $p(x)$ above to derive a term called Evidence Lower Bound(ELBO), which places a lower bound on the evidence- the log likelihood of the observed data. This allows us to define an objective for the latent variable model- to maximize the probability of the observed data indirectly, by maximizing the ELBO - the lower bound of the log of that probability. Below is the equation for the ELBO and its connection to evidence:
$$ \mathbb{E}_{q_{\phi}(z \mid x)} \left[ \log \frac{p_{\theta}(x, z)}{q_{\phi}(z \mid x)} \right] $$
$$$$
$$\log p(x) \ge \mathbb{E}_{q_{\phi}(z \mid x)} \left[ \log \frac{p_{\theta}(x, z)}{q_{\phi}(z \mid x)} \right] $$
$$$$
where we introduce $q_{\phi}(z \mid x)$ as a flexible approximate variational distribution with parameters $\phi$ instead of the original intractable distribution $p(z|x)$. The idea is that a model is learned to estimate the true intractable distribution of latent variables over given observations for $x$.
As the models is trained its parameters are optimized and its estimation of the true prior distribution becomes better and better. 

#### ELBO Derivation

To derive the ELBO we start with the expression for marginalizing the latent variable:

$$p(x) = \int p(x, z)dz$$
apply log on both sides,
$$ \log p(x) = \log \int p(x, z)dz$$
multiply the integrand by one in the form of $\frac{q_{\phi}(z \mid x)}{q_{\phi}(z \mid x)}$ to introduce the approximate variational distribution(the estimation of the intractable posterior learnt by a neural network)
$$ \log p_{\theta}(x) = \log \int q_{\phi}(z \mid x) \frac{p_{\theta}(x, z)}{q_{\phi}(z \mid x)} \, dz $$
Now, the expectation of a arbitrary deterministic expression $f(x)$ is given by the integral $$ \mathbb{E}[f(x)] = \int f(x) p(x) \, dx $$ therefore, the integral expression for the evidence can be rewritten as an expectation of the function $\frac{p_{\theta}(x, z)}{q_{\phi}(z \mid x)} $ under the distribution $ q_{\phi}(z \mid x) $:
$$ \log p(x) = \log \mathbb{E}_{q_{\phi}(z \mid x)} \left[ \frac{p_{\theta}(x, z)}{q_{\phi}(z \mid x)} \right] $$
Now, because of the logarithm's concavity and the fact that the log of an expectation is always greater than or equal to the expectation of the log (this is a consequence of Jensen's inequality):$$ \log \mathbb{E}[f(z)] \geq \mathbb{E}[\log f(z)] $$
we can bring the log inside the expectation:
$$ \log p(x) \ge  \mathbb{E}_{q_{\phi}(z \mid x)} \left  [ \log \frac{p_{\theta}(x, z)}{q_{\phi}(z \mid x)} \right] $$

Thus we have derived the lower bound of the evidence. Now we can explore the ELBO term further.

#### ELBO in More Detail

If we start with the equation of the ELBO:
$$\mathbb{E}_{q_{\phi}(z \mid x)} \left  [ \log \frac{p_{\theta}(x, z)}{q_{\phi}(z \mid x)} \right]$$
We can apply the chain rule of probability to substitute $p_{\theta}(x, z)$ with $p_{\theta}(x|z)p(z)$:
$$\mathbb{E}_{q_{\phi}(z \mid x)} \left  [ \log \frac{p_{\theta}(x|z)p(z)}{q_{\phi}(z \mid x)} \right]$$
Next, since expectation is linear and using logarithm rules we can split the expression like this:
$$\mathbb{E}_{q_{\phi}(z \mid x)} \left  [ \log p_{\theta}(x|z) \right] +  \mathbb{E}_{q_{\phi}(z \mid x)} \left  [ \log \frac{p_{}(z)}{q_{\phi}(z \mid x)} \right] $$

Here we can recognize the second term, it is Kullback–Leibler Divergence which provides a way measure if two distributions are close together or not. It can be though of as the distance between two distributions since like distance it is also always non-negative. There is a small caveat with this interpretation though, since unlike distance KL divergence is not symmetric. Namely the KL divergence between distributions A and B is not the same as B and A. The definition of KL divergence is:$$ D_{\text{KL}}(P \parallel Q) = \mathbb{E}_{x \sim P} \left[ \log \frac{p(x)}{q(x)} \right] $$

If we substitute x in the general distribution with $z$ we get the second term of the previous expression:
$$
\underset{\text{Reconstruction term}}{\mathbb{E}_{q_{\phi}(z \mid x)} \left[ \log p_{\theta}(x \mid z) \right]} 

\underset{\text{KL Divergence term}}{- D_{\text{KL}}(q_{\phi}(z \mid x) \parallel p(z))}
$$

#### Model Architecture

<div>
    <img src="./data/encoder_decoder.png" alt="Image" width="300">
</div>

We can gain a better intuition about this evidence expression if we look at the larger picture. The variational distribution $ q_{\phi}(z \mid x) $ is a neural network that is learned to approximate the hidden posterior distribution. This network is the encoder, it transforms inputs into distribution over possible latents. Similarly $p(x|z)$ is another network learnt to convert a given latent variable vector $z$ into an observation $x$. This second network is the decoder.<br>

Each term in the ELBO expression above measures the performance of one of these two networks:
* The reconstruction term measures the performance of the decoder, it ensures that the learned distribution is modeling effective latents that the original data can be generated from.
* The second term measures the performance of the encoder, it ensures that the learned variational distribution is as similar as possible to the prior belief held over the latent variables.

As already discussed, we want the trained model to maximize the probability of observed samples. Since, evidence is the log of that probability, maximizing the evidence lower bound accomplishes that goal. From the final expression of the ELBO it is evident that maximizing it is equivalent to maximizing the first (reconstruction) term and minimizing the second (KL Divergence) term.

#### The Reparameterizaion Trick

There is a major hurdle for training a model such as the one we outlined in previous paragraphs. Namely, to optimize the ELBO via backpropagation, we need gradients of the ELBO with respect to the encoder's parameters $\phi$. However, we cannot compute these gradients. This is because  the Evidence Lower Bound (ELBO) contains an expectation over the approximate posterior distribution $q_{\phi}(z \mid x)$. This expectation involves sampling latent variables $z$ during training. However, the sampling operation introduces stochasticity in the computation, which disrupts the computational graph. Information about gradients enccounters a bottleneck and cannot flow through a random sampling process. To define the problem in more rigorously, if we want to take the gradient w.r.t. $\theta$ of this expectation - $E_{p(z)} [f_{\theta}(z)]$, we can rewrite the expectation as an integral,then bring the gradient inside the integral and rewrite it as an expectation:

$$
\begin{align}
\nabla_{\theta} E_{p(z)} [f_{\theta}(z)] 
&= \nabla_{\theta} \left[ \int_{z} p(z) f_{\theta}(z) \, dz \right] \\
&= \int_{z} p(z) \left[ \nabla_{\theta} f_{\theta}(z) \right] \, dz \\
&= E_{p(z)} \left[ \nabla_{\theta} f_{\theta}(z) \right]
\end{align}
$$

We see that the gradient of the expectation is equal to the expectation of the gradient. However if we try to take the gradient with respect to $\theta$ we get:

$$
\begin{aligned}
\nabla_{\theta} E_{p_{\theta}(z)} [f_{\theta}(z)] 
&= \nabla_{\theta} \left[ \int_{z} p_{\theta}(z) f_{\theta}(z) \, dz \right] \\
&= \int_{z} \nabla_{\theta} \left[ p_{\theta}(z) f_{\theta}(z) \right] \, dz \\
&= \int_{z} f_{\theta}(z) \nabla_{\theta} p_{\theta}(z) \, dz 
+ \int_{z} p_{\theta}(z) \nabla_{\theta} f_{\theta}(z) \, dz \\
&= \underset{\text{(Problematic Term)}}{\int_{z} f_{\theta}(z) \nabla_{\theta} p_{\theta}(z) \, dz} 
+ E_{p_{\theta}(z)} \left[ \nabla_{\theta} f_{\theta}(z) \right]
\end{aligned}
$$

The first term of the last equation is not guaranteed to be an expectation, and we do not have an analytic solution to it.

The key idea to solve this proposed by the paper is to have the stochastic element not as part of the model, but as its input. This can be achieved if instead of sampling $z \sim q_\phi(z \mid x)$, $z$ can be rewritten  as a deterministic transformation of a noise variable $\epsilon$ sampled from a standard Gaussian distribution $N(0, I)$. For a latent variable $z \sim N(\mu, \sigma^2)$, this can be expressed as:

$$
z = g_{\phi}(x, \epsilon), \quad \epsilon \sim N(0, I)
$$

where
$$
g_{\phi}(x, \epsilon) = \mu_{\phi}(x) + \sigma_{\phi}(x) \odot \epsilon
$$

Now if we try the problematic differentiation again using $l \in L$ to denote the lth Monte Carlo sample we get:

$$
\begin{aligned}
E_{p_{\theta}(z)} [f(z^{(i)})] &= E_{p(\epsilon)} [f(g_{\theta}(\epsilon, x^{(i)}))] \\
\nabla_{\theta} E_{p_{\theta}(z)} [f(z^{(i)})] &= \nabla_{\theta} E_{p(\epsilon)} [f(g_{\theta}(\epsilon, x^{(i)}))] \\
&= E_{p(\epsilon)} \left[ \nabla_{\theta} f(g_{\theta}(\epsilon, x^{(i)})) \right] \\
&\approx \frac{1}{L} \sum_{l=1}^{L} \nabla_{\theta} f(g_{\theta}(\epsilon^{(l)}, x^{(i)}))
\end{aligned}
$$

When we use the reparameterization trick we can again express a gradient of an expectation as an expectation of a gradient.

<div style="text-align: center;">
    <img src="./data/vae.png" alt="Image" width="1000>
</div>

<div>
    <img src="./data/vae.png" alt="Image" width="1000">
</div>

#### Closed Form of KL Divergence

Under some assumptions the KL divergence term has a closed form, namely when:
* The prior $p(z)$ over the latent variables $z$ is a standard Gaussian $\mathcal{N}(0, I)$, i.e., $p(z) = \mathcal{N}(z|0, I)$.

* The variational posterior $ q(z|x)$ is also a Gaussian, typically with a diagonal covariance matrix, i.e., $q(z|x) = \mathcal{N}(z|\mu(x), \sigma(x)^2)$, where $\mu(x) $ and $\sigma(x)^2 $ are the outputs of the encoder network.


Under these conditions the closed form is :
$$
\begin{aligned}
\text{KL}(q(z|x) \| p(z)) &= \int q(z|x) \log \frac{q(z|x)}{p(z)} dz \\
&= \int \mathcal{N}(z|\mu(x), \sigma(x)^2) \log \frac{\mathcal{N}(z|\mu(x), \sigma(x)^2)}{\mathcal{N}(z|0, I)} dz \\
&= \int \mathcal{N}(z|\mu(x), \sigma(x)^2) \left( \log \mathcal{N}(z|\mu(x), \sigma(x)^2) - \log \mathcal{N}(z|0, I) \right) dz \\
&= \int \mathcal{N}(z|\mu(x), \sigma(x)^2) \left[ -\frac{1}{2} \left( \log 2\pi + \log \det(\sigma(x)^2) + (z - \mu(x))^T \sigma(x)^{-2} (z - \mu(x)) \right) + \frac{1}{2} \left( (z^T z) \right) \right] dz \\
\end{aligned}
$$

This is continuous theoretical form of the KL divergence for two Gaussian distributions. To be able to compute it we will need to discretize it to the following form:

$$
\begin{aligned}
\text{KL}(q(z|x) \| p(z)) 
&= \frac{1}{2} \left( \text{tr}(\sigma(x)^{-2}) + \mu(x)^2 - d - \log \det(\sigma(x)^2) \right)\\
&= \frac{1}{2} \left( \mu(x)^2 + \sigma(x)^2 - \log \sigma(x)^2 - 1 \right)\\
&= -\frac{1}{2} \left( 1  + \log \sigma(x)^2 - \mu(x)^2 - \sigma(x)^2  \right)
\end{aligned}
$$


### Loss Function

When exploring the ELBO in more detail we arrived at this expression:
$$
\underset{\text{Reconstruction term}}{\mathbb{E}_{q_{\phi}(z \mid x)} \left[ \log p_{\theta}(x \mid z) \right]} 

\underset{\text{KL Divergence term}}{- D_{\text{KL}}(q_{\phi}(z \mid x) \parallel p(z))}
$$

This is what we want to maximize to have a good generative model. However, in machine learning we usually use gradient descent to find a minimum for the loss function of the model. To stick with this approach we will be minimizing the negative ELBO. With this in mind we can the define the VAE loss functions as follows:

$$
L(\theta, \phi; x) = - \mathbb{E}_{q_{\phi}(z \mid x)} \left[ \log p_{\theta}(x \mid z) \right] + D_{\text{KL}}(q_{\phi}(z \mid x) \parallel p(z))
$$


### Forward Pass

Below we outline the forward pass of the model:
1. Push the input $x$ through the encoder:
    * The input $x$ is passed through the encoder network, which outputs the mean $\mu(x)$ and the standard deviation $\sigma(x)$ (or equivalently the log-variance)
    * $\mu(x), \sigma(x) = Encoder(x)$

2. Sample noise:
    * To allow backpropagation through the stochastic part of the network, we sample $\epsilon$ from a standard normal distribution $\epsilon \sim N(0,I)$
    * $\mu(x), \sigma(x) = Endoder(x)$

3. Reparametarize:
    * Using the reparameterization trick, the latent variable $z$ is sampled as:
    $$
    g_{\phi}(x, \epsilon) = \mu_{\phi}(x) + \sigma_{\phi}(x) \odot \epsilon
    $$
    * This allows the gradient to flow through $\mu(x)$ and  $\sigma(x)$ during backpropagation
    * In case  of using the log variance  $\log(\sigma^2)$ the formula for the standard deviation $ \sigma $ can be derived in this way

    -  Start with the logarithm of the variance:  
   $$ \text{logvar} = \log(\sigma^2) $$

    -  Exponentiate both sides to obtain the variance:  
   $$ \sigma^2 = \exp(\text{logvar}) $$

    -  Take the square root to get the standard deviation:  
   $$ \sigma = \sqrt{\sigma^2} = \sqrt{\exp(\text{logvar})} $$

    -  Use the property of exponents $\sqrt{\exp(a)} = \exp(a / 2) $:  
   $$ \sigma = \exp(0.5 \cdot \text{logvar}) $$

    -  Thus, the standard deviation is computed as:  
   $$ \sigma = \exp(0.5 \cdot \text{logvar}) $$

4. Push $z$ through the decoder:
    * The reparameterized latent variable $z$ is then passed through the decoder to reconstruct the input $x$, yielding a predicted $\bar{x}$
    * $\bar{x} = Decoder(z)$

5. Compute the reconstruction loss
    * The reconstruction loss measures how well the decoder reconstructs the input, often using Mean Square Error (MSE), but in some specific cases as with black and white images we can Binary Cross Entropy (BCE)

6. Compute the variational loss (KL divergence)
    * The variational loss measures the divergence between the approximate posterior $q_{\phi}(z \mid x) = \mathcal{N}(\mu(x), \sigma(x)^2)$ and the prior $p(z) = \mathcal{N}(0, I)$
    $$\text{Var. Loss} = D_{\text{KL}}[\mathcal{N}(\mu(x), \sigma(x)^2) \parallel \mathcal{N}(0, I)]$$

7. Combine the two loss terms for the final loss
    * The total loss for the VAE is the combination of the reconstruction loss and the variational loss:
    $$Total Loss=Reconstruction Loss + Variational Loss$$

### Intuition

An Autoencoder is a powerful concept on its own. It is a neural network that learns data representations in an unsupervised manner, without relying on labels. The network learns to compress data into a latent space in a way that allows it to be reconstructed as closely as possible to the original input. However, since it is trained using only reconstruction loss, there are no constraints placed on the structure of the latent space. This becomes problematic when we want to generate new, unseen samples from the latent space using only the decoder. In this case, the latent space lacks any inherent structure, which could result in irregularities like holes or discontinuities. If we try to generate a sample from a randomly chosen latent vector, there is no guarantee that it will correspond to a valid point in the data distribution, making the generation of realistic samples unreliable.

The Variational Autoencoder (VAE) addresses this issue by adding a new term to the objective function: the KL divergence term. This term acts as a regularizer, effectively pushing points in the latent space closer to the origin. By doing so, it eliminates gaps in the latent space and often enforces a semantic structure, where similar inputs are mapped to nearby points in the latent space. The VAE's objective function can be seen as a delicate balance: the reconstruction term attempts to expand the latent space, while the variational term seeks to compact it. In the vanilla VAE implementation, these two terms are simply added together to form the objective function. However, later improvements introduce parameters that weight the terms, providing finer control over what the model is optimizing. This approach allows us to seek a fine-tuned balance between reconstruction quality and latent space regularization.
